<a href="https://colab.research.google.com/github/domwelsh/avcad_final/blob/main/notebooks/Preparation_for_identification_model/Images_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process for PlanetScope Image Download

### Considerations:

1. The image was already downloaded from the PlanetScope repository and uploaded to Google Cloud Earth Engine Asset.
2. This was done to avoid the high time in calculating the indexes; it is more efficient to do this using geemap and/or Python.

### Index



In [ ]:
# Import the libraries
import geemap
import ee

In [ ]:
# Authenticate your GEE account
ee.Authenticate()


Successfully saved authorization token.


In [ ]:
# This step is relevant for could run the project.py file
# Importar librerias al entorno
import ee

# Iniciar Sesión en GEE (si les da un error incluso pegando el link de autenticación hay que conectar GEE con Google Cloud)
ee.Authenticate()
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
# Add a map to display the image and index
Map = geemap.Map(center = (39.22, -8.97), zoom = 6)
Map

Map(center=[39.22, -8.97], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
Map.user_roi.getInfo()
#Define the geometry of the study area
geometry = Map.user_roi

In [ ]:
Map.user_roi.getInfo()

{'type': 'Point', 'coordinates': [-8.607292, 38.134876]}

## The points of each downloaded area are:
1. Quercus_1: geometry = ee.Geometry.Point([-8.796394, 39.205319])
2. Quercus_2: geometry = ee.Geometry.Point([-8.82684, 38.964544])
3. Quercus_3: geometry = ee.Geometry.Point([-9.130675, 38.625997])
4. Quercus_4: geometry = ee.Geometry.Point([-7.688382, 38.655686])
5. Quercus_5: geometry = ee.Geometry.Point([-8.515074, 38.333295])
6. Quercus_6: geometry = ee.Geometry.Point([-8.548938, 38.175632])
7. Quercus_7: geometry = ee.Geometry.Point([-8.727648, 38.118924])
8. Quercus_8: geometry = ee.Geometry.Point([-8.73125, 38.092178])
9. Quercus_9: geometry = ee.Geometry.Point([-8.510705, 37.186572])
10. Quercus_10: geometry = ee.Geometry.Point([-8.691703, 37.92617])
11. Quercus_11: geometry = ee.Geometry.Point([-8.607292, 38.134876])

In [ ]:
# Asegurarnos de la creación de una geometría / Por si no la crearon en el mapa anterior.
geometry = ee.Geometry.Point([-8.607292, 38.134876])


In [ ]:
# Call the image collection
collection = ee.ImageCollection("projects/ee-jesusc461/assets/Planet_quercus/Quercus_11")

# Filter by date (in this case is the same month of soil sampling)
collection = collection.filterDate('2023-05-01', '2024-05-31').filterBounds(geometry)

# This function maps spectral indices for salinity Mapping using PlanetScope Imagery
def addIndices(img):
    # NDVI
    NDVI = img.normalizedDifference(['nir', 'red']).rename('NDVI')

    # SR (Simple Ratio)
    SR = img.select('nir').divide(img.select('red')).rename('SR')

    # GCVI (Green Chlorophyll Vegetation Index)
    GCVI = img.expression('(NIR / GREEN) - 1', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GCVI')

    # NDWI (Normalized Difference Water Index)
    NDWI = img.expression('(GREEN - NIR) / (GREEN + NIR)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('NDWI')

    # VARI (Visible Atmospherically Resistant Index)
    VARI = img.expression('(Green - Red) / (Green + Red - Blue)', {'Blue': img.select('blue'), 'Red': img.select('red'), 'Green': img.select('green')}).rename('VARI')

    # GRVI (Green Ratio Vegetation Index)
    GRVI = img.expression('(NIR / GREEN)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GRVI')

    # GNDVI (Green Normalized Difference Vegetation Index)
    GNDVI = img.normalizedDifference(['nir', 'green']).rename('GNDVI')

    # SAVI (Soil Adjusted Vegetation Index)
    SAVI = img.expression('(NIR - RED) / ((NIR + RED + 0.5) * 1.5)', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('SAVI')

    # ARVI (Atmospherically Resistant Vegetation Index)
    ARVI = img.expression(
        '(NIR - (RED - 1.0 * (RED - BLUE))) / (NIR + (RED - 1.0 * (RED - BLUE)))',
        {'NIR': img.select('nir'), 'RED': img.select('red'), 'BLUE': img.select('blue')}
    ).rename('ARVI')

    # EVI (Enhanced Vegetation Index)
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6.0 * RED - 7.5 * BLUE + 1.0))',
        {'NIR': img.select('nir'), 'RED': img.select('red'), 'BLUE': img.select('blue')}
    ).rename('EVI')

    # DVI (Difference Vegetation Index)
    DVI = img.expression('NIR - RED', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('DVI')

    return img.addBands([NDVI, SR, GCVI, NDWI, VARI, GRVI, GNDVI, SAVI, ARVI, EVI, DVI])

# Example usage
ps = collection
#Add the indices
ps = ps.map(addIndices)
composite = ps \
              .mean()

Map.addLayer(composite, {'bands': ['red',  'green',  'blue'], 'min': 201, 'max': 2464}, 'RGB')